In [1]:
# %load_ext autoreload
# %autoreload 2
import sys
sys.path.append("..")

In [2]:
import pandas as pd
import torch
import torch.nn as nn
import timm
from torch import optim
from utils.ProgressBar import ProgressBar
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, random_split
from MachineLearning.ApartmentsDatasetPyTorch import ApartmentsDatasetPyTorch
from tqdm import tqdm

In [3]:
params = {
    "data_dir" : "../processed_data/data.csv",
    "images_dir" : '../scraping_results/images',
    "img_input_size" : 256,
    "batch_size" : 256, # because we have a lot of columns
    "shuffle" : True,
    
    "inception_model_output_size" : 256,
    "tabular_ffnn_output_size" : 64,
    "learning_rate" : 1e-3,
    "weight_decay" : 1e-5
}

In [10]:
class ModifiedInception(nn.Module):
    def __init__(self, output_size):
        super(ModifiedInception, self).__init__()
        # Load and modify the Inception model
        self.inception = timm.create_model('inception_v4', pretrained=True)
        n_features = self.inception.last_linear.in_features
        self.inception.last_linear = nn.Linear(n_features, output_size) 

        # Freeze all layers of Inception model except last linear layer
        for param in self.inception.parameters():
            param.requires_grad = False
        for param in self.inception.last_linear.parameters():
            param.requires_grad = True

    def forward(self, x):
        return self.inception(x)

class TabularFFNN(nn.Module):
    def __init__(self, input_size, output_size, dropout_prob=0.3):
        super(TabularFFNN, self).__init__()
        self.ffnn = nn.Sequential(
            nn.Linear(input_size, 256),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm1d(256),  # Ensure the input here has 512 features
            nn.Linear(256, 256),
            nn.Dropout(dropout_prob),
            nn.Sigmoid(),
            nn.Linear(256, 256),
            nn.LeakyReLU(negative_slope=0.01),
            nn.Linear(256, 256),
            nn.Sigmoid(),
            nn.Dropout(dropout_prob),
            nn.Linear(256, 64),
            nn.LeakyReLU(negative_slope=0.01),
            nn.Linear(64, output_size)
        )

    def forward(self, x):
        x = x.float()
        x = x.squeeze(1)
        x = self.ffnn(x)
        return x

class RegressionModel(nn.Module):
    def __init__(self, input_size, dropout_prob = 0.5):
        super(RegressionModel, self).__init__()
        print(input_size)
        self.regression = nn.Sequential(
            nn.Linear(input_size, 256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(negative_slope=0.05),
            nn.Linear(256, 256),
            nn.LeakyReLU(negative_slope=0.05),
            nn.Dropout(dropout_prob),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.Dropout(dropout_prob),
            nn.ReLU(),
            nn.Linear(128, 1)  # Output layer for regression, no activation
        )

    def forward(self, x):
        x = self.regression(x)
            
        return x

class CustomCombinedModel(nn.Module):
    def __init__(self, tabular_data_size):
        super(CustomCombinedModel, self).__init__()
        # Inception
        inception_model_output_size = params["inception_model_output_size"]
        self.modified_inception = ModifiedInception(inception_model_output_size)
        
        # Tabular
        tabular_ffnn_output_size = params["tabular_ffnn_output_size"]
        self.tabular_ffnn = TabularFFNN(
            input_size = tabular_data_size,
            output_size = tabular_ffnn_output_size
        )
        
        self.regression_model = RegressionModel(
            input_size = inception_model_output_size + tabular_ffnn_output_size
        )  # 512 from Inception and 16 from TabularFFNN

        print("Inception output size", inception_model_output_size)
        print("Tabular output size", tabular_ffnn_output_size)
        print("Regression input size", inception_model_output_size + tabular_ffnn_output_size)
        
    def forward(self, image_tensor, tabular_data):
        image_output = self.modified_inception(image_tensor)
        tabular_output = self.tabular_ffnn(tabular_data)
        combined_output = torch.cat((image_output, tabular_output), dim=1)
        price = self.regression_model(combined_output)
        return price


In [11]:
transform = transforms.Compose([
    transforms.Resize((params["img_input_size"], params["img_input_size"])),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize images
])

dataset = ApartmentsDatasetPyTorch(
    data_dir = params["data_dir"],
    images_dir = params["images_dir"], 
    transform = transform
)

model = CustomCombinedModel(dataset.tabular_data_size())

gpu = torch.device("mps:0")
model = model.to(gpu)

320
Inception output size 256
Tabular output size 64
Regression input size 320


# Training

In [12]:
def custom_collate_fn(batch):
    batch = [data for data in batch if data[-1] == True]
    if not batch:
        return None
    images, datas, prices, is_valid = zip(*batch)
    images = torch.stack(images).to(gpu)
    datas = torch.stack(datas).to(gpu)
    prices = torch.stack(prices).to(gpu)
    
    return images, datas, prices

# Create the dataloader
dataloader = DataLoader(
    dataset,
    batch_size = params["batch_size"],
    shuffle = params["shuffle"],
    collate_fn=custom_collate_fn
)

In [ ]:
# Assuming 'dataset' is your full dataset
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size
num_epochs = 20
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])
num_GPU = 1

train_loader = DataLoader(
    train_dataset,
    batch_size=params["batch_size"], 
    shuffle=params["shuffle"], 
    collate_fn=custom_collate_fn,
    
    num_workers=4*num_GPU,
    pin_memory=True
)
val_loader = DataLoader(
    val_dataset,
    batch_size=params["batch_size"],
    shuffle=False, 
    collate_fn=custom_collate_fn,
    
    num_workers=4*num_GPU,
    pin_memory=True
)
test_loader = DataLoader(
    test_dataset,
    batch_size=params["batch_size"], 
    shuffle=False,
    collate_fn=custom_collate_fn,
    
    num_workers=4*num_GPU,
    pin_memory=True
)

# Initialize lists to track losses
train_losses = []
val_losses = []

optimizer = optim.Adam(
    model.parameters(), 
    lr=params["learning_rate"],
    weight_decay = params["weight_decay"]
)
criterion = torch.nn.MSELoss()

print("Starting training...")
# Training loop with progress bar
for epoch in range(num_epochs):
    # Training
    model.train()  # Set the model to training mode
    running_loss = 0.0
    for data in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs} - Training'):
        images, datas, prices = data
        optimizer.zero_grad()  # Zero the gradients
        outputs = model(images, datas)  # Forward pass
        loss = criterion(outputs, prices.view(-1, 1).float())  # Compute loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights

        running_loss += loss.item()  # Accumulate the loss

    train_losses.append(running_loss / len(train_loader))  # Average loss for this epoch

    # Validation
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    with torch.no_grad():  # Disable gradient calculation
        for data in tqdm(val_loader, desc=f'Epoch {epoch+1}/{num_epochs} - Validation'):
            images, datas, prices = data
            outputs = model(images, datas)  # Forward pass
            loss = criterion(outputs, prices.view(-1, 1).float())  # Compute loss
            val_loss += loss.item()  # Accumulate the loss

    val_losses.append(val_loss / len(val_loader))  # Average loss for this epoch

    # Print epoch's summary
    print(f'Epoch {epoch+1}, Training Loss: {train_losses[-1]}, Validation Loss: {val_losses[-1]}')


Starting training...


Epoch 1/20 - Training:  57%|██████████▊        | 69/121 [09:31<07:09,  8.27s/it]

In [ ]:
# torch.save(model, "model_v3.pth")

In [ ]:
# !conda install --yes pytorch torchvision -c pytorch